In [ ]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import matplotlib.pyplot as plt
from keras_tqdm import TQDMNotebookCallback
from npm3dGenerator import NPM3DGenerator, NPM3DGenerator_full
from models import build_point_net
import utils_training as u_t

In [ ]:
training_dataset = NPM3DGenerator(paths_to_keep = [1])
validation_dataset = NPM3DGenerator(paths_to_keep = [0])

In [ ]:
miou_metric = u_t.mIoU(training_dataset.n_classes)

model = build_point_net(input_shape = (training_dataset.n_points, training_dataset.n_channels),
                        output_shape = training_dataset.n_classes)
model.compile("adadelta", u_t.my_categorical_crossentropy, metrics = ["accuracy", miou_metric.miou])
#model.summary()

In [ ]:
n_epochs = 10
history = model.fit_generator(training_dataset, epochs = n_epochs,
                              validation_data = validation_dataset,
                              class_weight = training_dataset.class_weight,
                              verbose = 0, callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
u_t.plot_history(history)

In [ ]:
from utils import save_model
save_model(model, "trained_{}".format(training_dataset.n_channels))

In [ ]:
test_dataset = NPM3DGenerator(input_dir = "../Benchmark_MVA/test", train = False)
from utils import load_model
model = load_model("trained_{}".format(test_dataset.n_channels))
predictions = test_dataset.predict_point_cloud(model)
plt.hist(np.argmax(predictions, axis = -1))